# Module 4

### Query4.1 
Import the csv file of the stock which contained the Bollinger columns as well.

* Create a new column 'Call' , whose entries are - 
  * 'Buy' if the stock price is below the lower Bollinger band 
  * 'Hold Buy/ Liquidate Short' if the stock price is between the lower and middle Bollinger band 
  * 'Hold Short/ Liquidate Buy' if the stock price is between the middle and upper Bollinger band 
  * 'Short' if the stock price is above the upper Bollinger band
  
* Now train a classification model with the 3 bollinger columns and the stock price as inputs and 'Calls' as output. Check the accuracy on a test set. (There are many classifier models to choose from, try each one out and compare the accuracy for each)

* Import another stock data and create the bollinger columns. Using the already defined model, predict the daily calls for this new stock.


In [2]:
import seaborn as sns
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt
import warnings
warnings.filterwarnings('ignore')
%matplotlib notebook

In [3]:
df = pd.read_csv('Small_Cap/FORTIS.csv')
df['Average'] = pd.read_csv("week2 v.csv")
df['+2 SD'] = pd.read_csv('week2 vu.csv')
df['-2 SD'] = pd.read_csv('week2 vl.csv')
df = df.dropna()
df['Date'] = df['Date'].astype('datetime64')
df.set_index('Date',inplace =True)
df.head()

,Symbol,Series,Prev Close,Open Price,High Price,Low Price,Last Price,Close Price,Average Price,Total Traded Quantity,Turnover,No. of Trades,Deliverable Qty,% Dly Qt to Traded Qty,Average,+2 SD,-2 SD
Date,,,,,,,,,,,,,,,,,
2017-06-01,FORTIS,EQ,195.15,195.90,200.25,195.15,199.80,198.35,197.35,837394,1.652597e+08,6033,248194,29.64,199.871429,209.101539,190.641318
2017-06-02,FORTIS,EQ,198.35,200.05,201.60,191.75,193.20,192.95,195.33,2967855,5.797230e+08,15457,1540619,51.91,198.992857,208.358166,189.627548
2017-06-05,FORTIS,EQ,192.95,194.00,196.90,193.60,194.75,194.90,195.37,1688838,3.299437e+08,12741,654094,38.73,198.064286,206.110253,190.018319
2017-06-06,FORTIS,EQ,194.90,195.40,195.45,189.10,190.40,191.05,192.04,3708378,7.121618e+08,17057,2344287,63.22,197.003571,204.496077,189.511066
2017-06-07,FORTIS,EQ,191.05,191.10,198.70,191.10,194.50,194.05,195.63,3368728,6.590180e+08,21279,1095546,32.52,196.803571,204.461394,189.145748


In [4]:
df['Call'] = np.nan
df['Call'] = df['Call'].astype('str')

for i in range(len(df)):
    if df['Close Price'][i] < df['-2 SD'][i]:
        df['Call'][i] = 'Buy'
    elif df['Close Price'][i] > df['-2 SD'][i] and df['Close Price'][i] < df['Average'][i] :
        df['Call'][i] = 'Hold Buy/ Liquidate Short'
    elif df['Close Price'][i] < df['+2 SD'][i] and df['Close Price'][i] > df['Average'][i]:
        df['Call'][i] = 'Hold Short/ Liquidate Buy'
    elif df['Close Price'][i] > df['+2 SD'][i] :
        df['Call'][i] = 'Short'
    
df.head()

,Symbol,Series,Prev Close,Open Price,High Price,Low Price,Last Price,Close Price,Average Price,Total Traded Quantity,Turnover,No. of Trades,Deliverable Qty,% Dly Qt to Traded Qty,Average,+2 SD,-2 SD,Call
Date,,,,,,,,,,,,,,,,,,
2017-06-01,FORTIS,EQ,195.15,195.90,200.25,195.15,199.80,198.35,197.35,837394,1.652597e+08,6033,248194,29.64,199.871429,209.101539,190.641318,Hold Buy/ Liquidate Short
2017-06-02,FORTIS,EQ,198.35,200.05,201.60,191.75,193.20,192.95,195.33,2967855,5.797230e+08,15457,1540619,51.91,198.992857,208.358166,189.627548,Hold Buy/ Liquidate Short
2017-06-05,FORTIS,EQ,192.95,194.00,196.90,193.60,194.75,194.90,195.37,1688838,3.299437e+08,12741,654094,38.73,198.064286,206.110253,190.018319,Hold Buy/ Liquidate Short
2017-06-06,FORTIS,EQ,194.90,195.40,195.45,189.10,190.40,191.05,192.04,3708378,7.121618e+08,17057,2344287,63.22,197.003571,204.496077,189.511066,Hold Buy/ Liquidate Short
2017-06-07,FORTIS,EQ,191.05,191.10,198.70,191.10,194.50,194.05,195.63,3368728,6.590180e+08,21279,1095546,32.52,196.803571,204.461394,189.145748,Hold Buy/ Liquidate Short


Classification models used are:
* Logistic Regression
* KNN
* Kernel SVM
* Naive Bayes
* Decision Tree
* Random Forest

In [5]:
def accuracy(cm):
    num = cm.trace()
    deno = cm.sum()
    return num/deno

In [6]:
X = df.iloc[:,[7,-4,-3,-2]].values
y = df.iloc[:,-1:].values

from sklearn.preprocessing import LabelEncoder
lb = LabelEncoder()
y = lb.fit_transform(y)
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size =0.2,random_state = 5)

from sklearn.preprocessing import StandardScaler
sc_X = StandardScaler()
X_train = sc_X.fit_transform(X_train)
X_test = sc_X.transform(X_test)

from sklearn.metrics import confusion_matrix

In [7]:
#Logistic Rergression

from sklearn.linear_model import LogisticRegression
classifer_logi = LogisticRegression(random_state = 5)
classifer_logi.fit(X_train,y_train)

y_pred_logi = classifer_logi.predict(X_test)

cm_logi = confusion_matrix(y_test,y_pred_logi)
print("The accuracy of logistic model classifier is :" + str(accuracy(cm_logi)))

The accuracy of logistic model classifier is :0.9587628865979382


In [8]:
# KNN

from sklearn.neighbors import KNeighborsClassifier
classifer_KNN = KNeighborsClassifier(n_neighbors = 3)
classifer_KNN.fit(X_train,y_train)

y_pred_KNN = classifer_KNN.predict(X_test)

cm_KNN = confusion_matrix(y_test,y_pred_KNN)
print("The accuracy of KNN model classifier is :" + str(accuracy(cm_KNN)))

The accuracy of KNN model classifier is :0.9278350515463918


In [9]:
# Kernel SVM
from sklearn.svm import SVC
classifer_kSVC = SVC(kernel='rbf', random_state = 5)
classifer_kSVC.fit(X_train,y_train)

y_pred_kSVC = classifer_kSVC.predict(X_test)

cm_kSVC = confusion_matrix(y_test,y_pred_kSVC)
print("The accuracy of Kernel SVM model classifier is :" + str(accuracy(cm_kSVC)))

The accuracy of Kernel SVM model classifier is :0.9484536082474226


In [10]:
# Naive Bayes
from sklearn.naive_bayes import GaussianNB
classifer_NB = GaussianNB()
classifer_NB.fit(X_train,y_train)

y_pred_NB = classifer_NB.predict(X_test)

cm_NB = confusion_matrix(y_test,y_pred_NB)
print("The accuracy of Naive Bayes model classifier is :" + str(accuracy(cm_NB)))

The accuracy of Naive Bayes model classifier is :0.4948453608247423


In [11]:
# Decision Tree
from sklearn.tree import DecisionTreeClassifier
classifer_DT = DecisionTreeClassifier()
classifer_DT.fit(X_train,y_train)

y_pred_DT = classifer_DT.predict(X_test)

cm_DT = confusion_matrix(y_test,y_pred_DT)
print("The accuracy of Decision Tree model classifier is :" + str(accuracy(cm_DT)))

The accuracy of Decision Tree model classifier is :0.845360824742268


In [13]:
# Random Forest
from sklearn.ensemble import RandomForestClassifier
classifer_RF = RandomForestClassifier()
classifer_RF.fit(X_train,y_train)

y_pred_RF = classifer_RF.predict(X_test)

cm_RF = confusion_matrix(y_test,y_pred_RF)
print("The accuracy of Random Forest model classifier is :" + str(accuracy(cm_RF)))

The accuracy of Random Forest model classifier is :0.9381443298969072


In [14]:
print('The most preffered model is Logistic Regression model as it has the highest accuracy')

The most preffered model is Logistic Regression model as it has the highest accuracy


In [33]:
df1 = pd.read_csv("Small_Cap/PVR.csv")
df1['Average'] = df1['Close Price'].rolling(14).mean()
df1['+2 SD'] = df1['Average'] + 2*df1['Close Price'].rolling(14).std()
df1['-2 SD'] = df1['Average'] - 2*df1['Close Price'].rolling(14).std()
df1 = df1.dropna()
df1['Date'] = df1['Date'].astype('datetime64')
df1.set_index('Date',inplace = True)

X_df1 = df1.iloc[:,[7,-3,-2,-1]].values
X_df1

array([[1455.7       , 1486.73928571, 1551.5432407 , 1421.93533072],
       [1457.8       , 1481.94285714, 1544.46396567, 1419.42174862],
       [1442.25      , 1475.03214286, 1531.47894023, 1418.58534548],
       ...,
       [1758.45      , 1743.46785714, 1832.4289602 , 1654.50675409],
       [1715.4       , 1743.36428571, 1832.46215264, 1654.26641879],
       [1724.        , 1744.36071429, 1832.15615992, 1656.56526865]])

In [34]:
y_df1_pred = classifer_logi.predict(X_df1)
y_df1_pred = lb.inverse_transform(y_df1_pred)
print(y_df1_pred)

['Hold Buy/ Liquidate Short' 'Hold Buy/ Liquidate Short'
 'Hold Buy/ Liquidate Short' 'Hold Buy/ Liquidate Short'
 'Hold Buy/ Liquidate Short' 'Hold Buy/ Liquidate Short'
 'Hold Buy/ Liquidate Short' 'Hold Buy/ Liquidate Short'
 'Hold Buy/ Liquidate Short' 'Hold Buy/ Liquidate Short'
 'Hold Buy/ Liquidate Short' 'Hold Buy/ Liquidate Short'
 'Hold Buy/ Liquidate Short' 'Hold Buy/ Liquidate Short'
 'Hold Buy/ Liquidate Short' 'Hold Buy/ Liquidate Short'
 'Hold Buy/ Liquidate Short' 'Hold Buy/ Liquidate Short'
 'Hold Buy/ Liquidate Short' 'Hold Buy/ Liquidate Short'
 'Hold Buy/ Liquidate Short' 'Hold Buy/ Liquidate Short'
 'Hold Buy/ Liquidate Short' 'Hold Buy/ Liquidate Short'
 'Hold Buy/ Liquidate Short' 'Hold Buy/ Liquidate Short'
 'Hold Buy/ Liquidate Short' 'Hold Buy/ Liquidate Short'
 'Hold Buy/ Liquidate Short' 'Hold Buy/ Liquidate Short'
 'Hold Buy/ Liquidate Short' 'Hold Buy/ Liquidate Short'
 'Hold Buy/ Liquidate Short' 'Hold Buy/ Liquidate Short'
 'Hold Buy/ Liquidate Short' 'H

Now, we'll again utilize classification to make a trade call, and measure the efficiency of our trading algorithm over the past two years. For this assignment , we will use RandomForest classifier.
* Import the stock data file of your choice
  Define 4 new columns , whose values are: 
  % change between Open and Close price for the day 
  % change between Low and High price for the day 
  5 day rolling mean of the day to day % change in Close Price 
  5 day rolling std of the day to day % change in Close Price
* Create a new column 'Action' whose values are: 
  1 if next day's price(Close) is greater than present day's. 
  (-1) if next day's price(Close) is less than present day's. 
  i.e. Action [ i ] = 1 if Close[ i+1 ] > Close[ i ] 
  i.e. Action [ i ] = (-1) if Close[ i+1 ] < Close[ i ]
* Construct a classification model with the 4 new inputs and 'Action' as target
* Check the accuracy of this model , also , plot the net cumulative returns (in %) if we were to follow this algorithmic model

In [30]:
len(y_test)

97

In [34]:
print(cm.shape)

(4, 4)


In [29]:
df1

,Symbol,Series,Prev Close,Open Price,High Price,Low Price,Last Price,Close Price,Average Price,Total Traded Quantity,Turnover,No. of Trades,Deliverable Qty,% Dly Qt to Traded Qty,Average,+2 SD,-2 SD
Date,,,,,,,,,,,,,,,,,
2017-06-01,PVR,EQ,1454.95,1452.00,1469.70,1452.00,1455.00,1455.70,1459.95,59522,8.689907e+07,3090,23335,39.20,1486.739286,1551.543241,1421.935331
2017-06-02,PVR,EQ,1455.70,1457.30,1466.90,1452.35,1460.00,1457.80,1462.45,55880,8.172155e+07,2865,29238,52.32,1481.942857,1544.463966,1419.421749
2017-06-05,PVR,EQ,1457.80,1458.90,1464.65,1440.00,1444.00,1442.25,1452.10,102733,1.491781e+08,4617,51065,49.71,1475.032143,1531.478940,1418.585345
2017-06-06,PVR,EQ,1442.25,1445.00,1447.45,1378.85,1390.00,1392.85,1411.78,210916,2.977667e+08,16465,89785,42.57,1465.214286,1527.738588,1402.689984
2017-06-07,PVR,EQ,1392.85,1391.10,1409.00,1387.00,1400.00,1400.70,1400.17,165824,2.321824e+08,7911,70517,42.53,1457.046429,1521.376556,1392.716301
2017-06-08,PVR,EQ,1400.70,1400.70,1498.50,1382.30,1461.00,1454.00,1465.71,596919,8.749125e+08,21846,142853,23.93,1452.792857,1508.298960,1397.286754
2017-06-09,PVR,EQ,1454.00,1465.00,1479.85,1443.00,1470.00,1472.25,1471.65,403598,5.939540e+08,18344,241292,59.79,1452.167857,1506.490209,1397.845506
2017-06-12,PVR,EQ,1472.25,1477.00,1507.00,1470.00,1495.00,1488.95,1492.22,306349,4.571401e+08,17326,98821,32.26,1451.632143,1504.192681,1399.071605
2017-06-13,PVR,EQ,1488.95,1490.55,1505.00,1474.00,1500.00,1496.30,1489.41,195198,2.907297e+08,8620,101610,52.05,1454.939286,1512.633215,1397.245356
